In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from lobster.model import LobsterPCLM
from lobster.model.llama_base._llama_encoder import LlamaEncoderModel

from transformers.models.llama.modeling_llama import LlamaConfig

import peft

In [ ]:
clm = LobsterPCLM(model_name='CLM_24M')

In [ ]:
clm.model.save_pretrained('clm_mini')

In [ ]:
clm.num_trainable_parameters

In [ ]:
c = LlamaConfig()

In [ ]:
m = LlamaEncoderModel.from_pretrained('clm_mini')

In [ ]:
m.num_parameters()

In [ ]:
fv_light = 'DIQMTQSPSSLSASVGDRVTITCRASQGIRNCLAWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLNGSSLQPEDFATYYCQQYNIYPLTFGQGTKVEIK'

In [ ]:
input_ids = torch.tensor([clm.tokenizer.encode(list(fv_light))])

In [ ]:
output = m.forward(input_ids=input_ids,
                  output_hidden_states=True)

In [ ]:
output.hidden_states[-1].shape

In [ ]:
peft.TaskType.

In [ ]:
peft_config = peft.LoraConfig(peft_type="LORA",
                              inference_mode=False,
                              r=16,  # dim of low rank matrices
                              lora_alpha=32,
                              lora_dropout=0.05,
                             target_modules= [
    "down_proj",
    "gate_proj",
    "q_proj",
    "o_proj",
    "up_proj",
    "v_proj",
    "k_proj"
  ],)

In [ ]:
peft_model = peft.PeftModel(m, peft_config)

In [ ]:
peft_model.print_trainable_parameters()